# Tutorial

## Installation

In [ ]:
!pip install ru_attacker

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/ruRoberta-large.zip"
!unzip "/content/drive/My Drive/TERRa.zip"

## Usage example

Set model

In [5]:
from ru_attacker.models import RobertaModel
roberta_checkpoints = "Roberta_checkpoints"
ruRoberta = RobertaModel(roberta_checkpoints)

Set dataset

In [6]:
from ru_attacker.models.set_dataset import get_data
data_dir = "TERRa/val.jsonl" 
data = get_data(data_dir)

Set attack

In [7]:
from ru_attacker.attacks import WordOrder
word_order_attack = WordOrder()

Unzipping /tmp/tmp3pfpl5f2.zip to /root/.cache/language_tool_python.
Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


Attack model and view results

In [8]:
results = word_order_attack.attack(ruRoberta, data)


                  [Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:
                  entailment --> not_entailment
                  original premise: """Решение носит символический характер, так как взыскать компенсацию практически невозможно"", - отмечается в сообщении."
                  original hypothesis: Взыскать компенсацию не получится.
                  
                  transformed: . получится Взыскать не компенсацию
                  
                  

                  [Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:
                  entailment --> entailment
                  original premise: Об этом вечером во вторник, 17 января, сообщила пресс-служба Спасательного департамента, отметив, что немецкую противотанковую мину Tellermine 42 обнаружили в на улице Кеэвисе в ходе земляных работ. Спасатели эвакуировали жителей окрестных домов, офисов и складских помещений. Уничтожать мину на месте не стали, поскольку это угрожало повреждению трассы трубопровода.
 

Convert results to DataFrame

In [9]:
import pandas as pd
dataframe = pd.DataFrame(results)
dataframe.head()

,original label,attacked label,original premise,original hypothesis,transformed,attack
0,1,0,"""""""Решение носит символический характер, так к...",Взыскать компенсацию не получится.,. получится Взыскать не компенсацию,failed
1,1,1,"Об этом вечером во вторник, 17 января, сообщил...",На улице Кеэвисе жителей эвакуировали из-за мины.,. из-за На Кеэвисе жителей улице мины эвакуиро...,succeeded
2,1,1,"""В суде Центрального района Сочи завершилось р...",81-летний пенсионер застрелил обидчика дочери.,. застрелил обидчика пенсионер 81-летний дочери,succeeded
3,1,1,"""Кто-то нарисовал на снегу контур человеческог...",Кто-то изобразил на снегу контур человеческого...,. снегу Кто-то тела изобразил человеческого на...,succeeded
4,1,0,"Также установлено, что работники учреждения не...",В детском саду не все вовремя проходят медосмотр.,не саду вовремя все В медосмотр детском . прох...,failed


## Create your own Attack or Model

Now lets create an attack that changes all character `ч` to `4` and `о` to `0`. For example, `Второй тайм резко отличался от первого.` --> `Вт0р0й
тайм резк0 0тли4ался 0т перв0г0.`

In [14]:
from ru_attacker.attacks import BasicAttack

class CharAttack(BasicAttack):
    def __init__(self):
        self.di = {"ч": "4", "о": "0"}

    def attack(self, model, dataset):
        transformations = []
        for i, row in dataset.iterrows():
            premise = row["premise"]
            hypothesis = row["hypothesis"]
            label = row["label"]
            prediction = model.predict(model.prepare_data(premise, hypothesis))
            if label == prediction:
                transformed = "".join([self.di[ch] if ch in self.di else ch for ch in hypothesis])
                prediction = model.predict(model.prepare_data(transformed, hypothesis))
                if label != prediction:
                    transformations.append(transformed)
        return transformations

In [15]:
char_attack = CharAttack()

In [16]:
transforms = char_attack.attack(ruRoberta, data)

Now let's look at this transformations

In [17]:
transforms

['Женщина ушла без заявления.',
 'Авария пр0из0шла в лесу.',
 'Вт0р0й тайм резк0 0тли4ался 0т перв0г0.',
 'В Р0ст0вск0й 0бласти п0дс4итали ут0нувших на Крещение.',
 'Неп0далеку застрял авт0м0биль их п0други.',
 'Незак0нные п0стр0йки устанавливать перестали.',
 'Сл0весная перебранка завершилась стрельб0й.',
 'Не кажд0му из пассажир0в п0л0жена к0мпенсация.',
 'У муж4ины в0сстан0вил0сь пле40.',
 'Специалисты перес4итают п0г0л0вье птиц другим сп0с0б0м.',
 'С0вы уже зд0р0вы.',
 'Семьи уже переехали из аварийных д0м0в в н0вые квартиры.',
 'Разрешение на стр0ительств0 у мэрии г0р0да п0лу4ить не удал0сь.',
 'В Евр0пе нав0днение.',
 'С0бака п0пала в пр0рубь.',
 'Четыре 0стальных магазина раб0тают без лицензии.',
 'В Самарск0й 0бласти с0хранится прежнее расписание электри4ек.',
 'Г0сударственная канцелярия выступила перед у4редителем.',
 'Мать не успела выпить спиртн0г0.',
 'Губернат0р не 0бещал материальную п0м0щь.',
 'Стражи п0рядка 0см0трели ф0рт04ку.',
 'У меня не 0стал0сь даже футб0ла.',
 '

Feel free to attack your own model, using `BasicModel` or any of implemented classes, or construct your own attacks with the help of `BasicAttack`!